# BICYCLE SALES ANALYSIS
A few examples of T-SQL queries for exploratory data analysis of a MS SQL Server database.
### Dataset
The dataset used is **Bicycle Sales** provided by DataCamp. It contains data on bicycle sales over several years in 

## EXPLORATORY QUERIES
### Dataset structure
Firstly, let's see what schemas, tables and fields are in the dataset, using the following SQL query:

In [4]:
SELECT TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS

,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE
0,production,categories,category_id,int
1,production,categories,category_name,varchar
2,production,brands,brand_id,int
3,production,brands,brand_name,varchar
4,production,products,product_id,int
5,production,products,product_name,varchar
6,production,products,brand_id,int
7,production,products,category_id,int
8,production,products,model_year,smallint
9,production,products,list_price,decimal


## Products
We can begin the analysis with identifying products and product categories which generate the highest revenue.

In [38]:
SELECT TOP 10
	soi.product_id,
	pp.product_name,
	SUM ((soi.quantity * soi.list_price) * (1 - soi.discount)) AS order_value
FROM sales.order_items AS soi
LEFT JOIN production.products AS pp
ON soi.product_id = pp.product_id
GROUP BY soi.product_id, pp.product_name
ORDER BY order_value DESC;

,product_id,product_name,order_value
0,7,Trek Slash 8 27.5 - 2016,555558.6111
1,9,Trek Conduit+ - 2016,389248.7025
2,4,Trek Fuel EX 8 29 - 2016,368472.7294
3,11,Surly Straggler 650b - 2016,226765.5510
4,56,Trek Domane SLR 6 Disc - 2017,211584.6153
5,10,Surly Straggler - 2016,203507.6200
6,8,Trek Remedy 29 Carbon Frameset - 2016,203380.8701
7,61,Trek Powerfly 8 FS Plus - 2017,188249.6235
8,58,Trek Madone 9.2 - 2017,175899.6482
9,51,Trek Silque SLR 8 Women's - 2017,174524.7315


In [72]:
SELECT
	pc.category_name,
	ROUND(SUM((soi.quantity * soi.list_price) * (1 - soi.discount)),2) AS order_value
FROM sales.order_items AS soi
LEFT JOIN production.products AS pp
ON soi.product_id = pp.product_id
LEFT JOIN production.categories AS pc
ON pp.category_id = pc.category_id
GROUP BY pc.category_name
ORDER BY order_value DESC;

,category_name,order_value
0,Mountain Bikes,2715079.53
1,Road Bikes,1665098.49
2,Cruisers Bicycles,995032.62
3,Electric Bikes,916684.78
4,Cyclocross Bicycles,711011.84
5,Comfort Bicycles,394020.10
6,Children Bicycles,292189.20


### Quantities in stock
We can also check if the revenue is correlated with the quantities in stock. In theory, one might expect that the highest stock quanities would belong to products that bring the most revenue (in this case, Mountain Bikes and Road Bikes).

To do that, I used a modified version of the previous query (introduced RANK window function) and wrapped it in a Common Table Expression. The resulting CTE is joined to a new query that checks the stock quantities per product.

In [49]:
WITH revenue AS (
	SELECT
		pc.category_name,
		RANK() OVER(ORDER BY SUM((soi.quantity * soi.list_price) * (1 - soi.discount)) DESC) AS sales_rank
	FROM sales.order_items AS soi
	LEFT JOIN production.products AS pp
	ON soi.product_id = pp.product_id
	LEFT JOIN production.categories AS pc
	ON pp.category_id = pc.category_id
	GROUP BY pc.category_name
)

SELECT
	pc.category_name AS category,
	SUM(ps.quantity) AS stock,
	cte.sales_rank
FROM production.products AS pp
LEFT JOIN production.stocks AS ps
ON pp.product_id = ps.product_id
LEFT JOIN production.categories AS pc
ON pp.category_id = pc.category_id
LEFT JOIN revenue AS cte
ON pc.category_name = cte.category_name
GROUP BY pc.category_name, cte.sales_rank
ORDER BY stock DESC;

,category,stock,sales_rank
0,Cruisers Bicycles,3378,3
1,Mountain Bikes,2654,1
2,Children Bicycles,2608,7
3,Road Bikes,2091,2
4,Comfort Bicycles,1258,6
5,Electric Bikes,1108,4
6,Cyclocross Bicycles,414,5


Interestingly, Children Bikes are at third place in terms of stock quantities, while generating the lowest overall revenue.

## Stores and orders
How ofter orders are delivered late across the organization? To answer that, I wrote a query to count the number of orders per number of days passed between the required date of delivery and actual date of shipment. If the number of days has a negative value, it means that the order was not shipped in time.

In [1]:
SELECT 
	shipment, 
	days_before_required, 
	COUNT(*) AS frequency 
FROM (
	SELECT
		DATEDIFF(day, shipped_date, required_date) AS days_before_required,
		(CASE WHEN 
	 		DATEDIFF(day, shipped_date, required_date) < 0 THEN 'delayed'
			ELSE 'on time' END) AS shipment
	FROM sales.orders
	WHERE shipped_date IS NOT NULL	-- we have to exclude currently processed orders
	) AS a
GROUP BY a.shipment, a.days_before_required;

,shipment,days_before_required,frequency
0,delayed,-2,153
1,delayed,-1,305
2,on time,0,505
3,on time,1,331
4,on time,2,151


Next, I check the percentage of orders that were delayed, per store.

In [71]:
SELECT
	ss.store_name,
	SUM(CASE WHEN DATEDIFF(day, so.shipped_date, so.required_date) < 0 THEN 1 END) AS orders_delayed,
	SUM(CASE WHEN DATEDIFF(day, so.shipped_date, so.required_date) >= 0 THEN 1 END) AS orders_on_time,
	SUM(CASE WHEN DATEDIFF(day, so.shipped_date, so.required_date) < 0 THEN 1 ELSE 0 END) * 100 / COUNT(so.shipped_date) AS delayed_percentage
FROM sales.stores AS ss
LEFT JOIN sales.orders AS so
ON ss.store_id = so.store_id
WHERE so.shipped_date IS NOT NULL
GROUP BY ss.store_name;

,store_name,orders_delayed,orders_on_time,delayed_percentage
0,Baldwin Bikes,317,702,31
1,Rowlett Bikes,37,105,26
2,Santa Cruz Bikes,104,180,36


Santa Cruz has the highest ratio of delayed orders to all orders.

## Customers
The following query returns the top 10 customers in terms of generated revenue.

In [1]:
SELECT TOP 10
	CONCAT(sc.first_name, ' ', sc.last_name) AS customer,
	sc.city,
	sc.state,
	COUNT(so.order_id) AS orders_made,
	ROUND(SUM((soi.quantity * soi.list_price) * (1 - soi.discount)),2) AS revenue
FROM sales.customers AS sc
LEFT JOIN sales.orders AS so
ON sc.customer_id = so.customer_id
LEFT JOIN sales.order_items AS soi
ON so.order_id = soi.order_id
GROUP BY CONCAT(sc.first_name, ' ', sc.last_name), sc.state, sc.city
ORDER BY revenue DESC;

,customer,city,state,orders_made,revenue
0,Sharyn Hopkins,Baldwinsville,NY,10,34807.94
1,Pamelia Newman,Monroe,NY,11,33634.26
2,Abby Gamble,Amityville,NY,7,32803.01
3,Lyndsey Bean,Fairport,NY,11,32675.07
4,Emmitt Sanchez,New York,NY,12,31925.89
5,Melanie Hayes,Liverpool,NY,7,31913.69
6,Debra Burks,Orchard Park,NY,11,27888.18
7,Elinore Aguilar,San Angelo,TX,10,25636.45
8,Corrina Sawyer,Troy,NY,8,25612.70
9,Shena Carter,Howard Beach,NY,4,24890.62
